In [ ]:
# General
from qutip import *
import pennylane as qml
import numpy as np
import numpy.ma as ma
import scipy as sci
import random
import math
import json
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import timeit
import checkLUE_upto_step22 as checkLUE
import sympy
import itertools
import mod_GGMM
from NC_LUE import NC
import networkx as nx

# For Defining Initial Objects
import qutipHam
import total_sys_GGMMs as ggmm
import mod_initstates as init

# For Scrambling Hamiltonians
import matplotlib.pyplot as plt
import mod_timeevol as te
import mod_score as sc

# Plotting parameters 
rcParams['figure.figsize'] = 8,8
u = 25
y_ub = 0.4
y_lb = -10**-2
ms = 30
opacity = 0.03
lineColor = 'black'
s1_args = {'s': ms, 'color': 'magenta', 'label': r'$\langle S_{\Pi} \rangle_L$', 'alpha': 1}
s1M_args = {'s': ms, 'color': 'purple', 'label': r'$\langle S_{\Pi} \rangle_L$', 'alpha': 1}
s2_args = {'s': ms, 'color': 'orchid', 'label': '$S_\Pi$', 'alpha': opacity}
s2M_args = {'s': ms, 'color': 'navy', 'label': '$S_{TPS}$', 'alpha': 1}
ze_args = {'color': lineColor, 'linestyle': '--'}

l1M_args = {'s': ms, 'color': 'lightskyblue', 'label': '$1$-local', 'alpha': 1}
l2M_args = {'s': ms, 'color': 'navy', 'label': '$2$-local', 'alpha': 1}
l3M_args = {'s': ms, 'color': 'thistle', 'label': '$3$-local', 'alpha': 1}
l4M_args = {'s': ms, 'color': 'darkviolet', 'label': '$4$-local', 'alpha': 1}

In [ ]:
num_env_qbs = 1
n = num_env_qbs+1
dim_sys = 2
dim_env = 2**num_env_qbs
dim_tot = 2**(n)
cc = 1
cx = cy = cz = 1
LU_or_U = 'y'

initial_hams = [qutipHam.H_ising(n, cc), qutipHam.H_sb(n, cc), qutipHam.H_heis(n, cx, cy, cz)]
ham_names = [r'$H_{is}$', r'$H_{sb}$', r'$H_{heis}$']

ham_num = 2
initial_ham = initial_hams[ham_num]
ham_name = ham_names[ham_num]
tensor_initial_states, initial_states = init.init_states(num_env_qbs)

# even superposition environment ready state
sysRho = np.zeros((2, 2))
sysRho[0][0] = 1
sysRho = Qobj(sysRho)

#random dm
randRho = rand_dm(2**n, dims = [[2]*n]*2)
randRho = [randRho]*2

envRho = Qobj(np.ones(shape=(2**(n-1), 2**(n-1))), dims = [[2]*(n-1)]*2)/2**(n-1)
eSup = tensor(sysRho, envRho)
eSup = [eSup, eSup]

# Constructing GGMMs
selfGGMMs = ggmm.constructSelfGGMMs(n, 0)
allGGMMs = ggmm.constructTotalGGMMs(n)

if LU_or_U == 'n':
    totGGMMs = selfGGMMs
    el = 3
else:
    totGGMMs = allGGMMs
    el = dim_tot**2-1

initial_thetas = [0]*len(totGGMMs)

In [ ]:
def haar_SU(n):
    '''
    Starts with haar unitary from qutip, then removes phase so determinant
    of resulting unitary is 1.
    '''
    N = 2**n
    rU = rand_unitary_haar(N, [[2]*n, [2]*n])
    return rU/(np.linalg.det(rU.full())**(1/N))

def HSIP(matrix1, matrix2, nVar):
    '''
    Hilbert Shmidt inner product for Hermitian matrices only!
    '''
    return ((matrix1.dag()*matrix2).tr())/2**nVar

def coeffGGMM(rho, GGMMs, nVar):
    '''
    Inputs:
    - rho: Hermitian matrix
    - n: Number of qubits composing the system
    - GGMMs: Generalized Gell-Mann Matrices
    Outputs:
    - coeff: Coefficients for vector representation (in GGMM basis) of M
    '''
    GGMM = GGMMs.copy()
    #GGMM.insert(0, tensor([qeye(2)]*n))
    coeff = [0]*len(GGMM)
    for i in range(len(GGMM)):
        coeff[i] = HSIP(rho, GGMM[i], nVar) 
    return coeff

def selfGGMMs(_n):
    lc = []
    _id = [qeye(2)]*_n
    _σ = [sigmax(), sigmay(), sigmaz()]
    _X = [sigmax()] + [qeye(2)]*(_n-1)
    _Y = [sigmay()] + [qeye(2)]*(_n-1)
    _Z = [sigmaz()] + [qeye(2)]*(_n-1)
    
    for i in range(3):
        _σi = _σ[i]
        for i in range(_n):
            term = _id.copy()
            term[i] = _σi
            lc.append(tensor(term))
    return lc

def selfCoeffs(_H, _GGMMs, _n):
    _selfGGMMs = selfGGMMs(_n)
    coeff = [0]*len(_selfGGMMs)
    for i in range(len(_selfGGMMs)):
        coeff[i] = HSIP(_H, _selfGGMMs[i], _n) 
    return coeff

def nonSelfCoeffs(_H, _GGMMs, _n):
    _nonSelfGGMMs = _GGMMs.copy()
    _selfGGMMs = selfGGMMs(_n)
    for x in _selfGGMMs:
        _nonSelfGGMMs.remove(x)
    coeff = [0]*len(_nonSelfGGMMs)
    for i in range(len(_nonSelfGGMMs)):
        coeff[i] = HSIP(_H, _nonSelfGGMMs[i], _n) 
    return coeff

def randomThetas(LUorU):
    lub = math.pi/4
    if LUorU == 0:
        scram_thetas = np.random.uniform(-lub, lub, el)
        return scram_thetas
    if LUorU == 1:
        scram_thetas = np.random.uniform(-lub, lub, 3*n)
        return scram_thetas
    else:
        return print('Error: entered LU value that is not 1 or 0.')

def scram(nVar, hamiltonian, thetasVar, GGMMs, LUorU):
    '''
   Inputs:
    - hamiltonian: Hamiltonian you want to scramble
    - thetas: Theta parameters corresponding to each GGMMs (LU = 0) or 
      to each selfGGMMs set (LU = 1)
   Outputs:
    - H_scram: Scrambled Hamiltonian 
    '''
    if LUorU == 0:
        #U = haar_SU(nVar)
        U = sc.construct_unitary(nVar, thetasVar, GGMMs)
        return U*hamiltonian*U.dag()
    if LUorU == 1:
        LU = sc.construct_localUnitary(nVar, thetasVar)
        return LU * hamiltonian * LU.dag()
    else:
        return print('Error: entered LU value that is not 1 or 0.')

def permuteHam(nVar, hamiltonian):
    '''
    Outpus:
    - allHams: list of original hamiltonian with its system permutations.
    '''
    allHams = [hamiltonian]
    for i in range(1, nVar):
        permuteList = list(range(nVar))
        permuteList[0], permuteList[i] = permuteList[i], permuteList[0]
        permHam = hamiltonian.permute(permuteList)
        allHams.append(permHam)
    return allHams

def APHam(nVar, hamiltonian):
    # System permutations
    allHams = permuteHam(nVar, hamiltonian)
    # Non system permutations 
    nCombs = math.comb(nVar, 2) - nVar + 1
    for i in range(len(allHams)):
        for j in range(nCombs):
            permuteList = list(range(nVar))
            combi = list(itertools.combinations(permuteList[1:], 2))
            permuteList[combi[j][0]], permuteList[combi[j][1]] = permuteList[combi[j][1]], permuteList[combi[j][0]]
            permHam = allHams[i].permute(permuteList)
            allHams.append(permHam)
    return allHams

def cost(thetasVar, nVar, hamiltonian, GGMMs, tensor_initial_statesVar, LUorU, OneorTwo):
    allScores = []
    scramHamVar = scram(nVar, hamiltonian, thetasVar, GGMMs, LUorU)
    allHams = permuteHam(nVar, scramHamVar)
    # Cost function 1
    if OneorTwo == 1:
        #for i in range(len(allHams)):
        allRhos = permuteHam(nVar, tensor_initial_statesVar[0])
        #tau = te.characteristic_time(allHams[i])
        tau = te.characteristic_time(scramHamVar)
        TEO = (-1j*scramHamVar*tau).expm()
        scores = []
        #for j in range(len(tensor_initial_statesVar)):
            #rhoTau = TEO*tensor_initial_statesVar[j]*TEO.dag()
            #rhoTauRed = rhoTau.ptrace(0)
            #score = 1 - (rhoTauRed*rhoTauRed).tr()
            #scores.append(score)
        for j in range(len(allRhos)):
            rhoTau = TEO*allRhos[j]*TEO.dag()
            rhoTauRed = rhoTau.ptrace(j)
            score = 1 - (rhoTauRed*rhoTauRed).tr()
            scores.append(score)
        return max(scores)
        #return scores
        #score = max(scores)
        #allScores.append(score)
        #return max(allScores)
    # Cost function 2
    if OneorTwo == 2:
        sigma = [sigmax(), sigmay(), sigmaz()]
        I_env = [qeye(2)]*(nVar - 1)
        for j in range(len(allHams)):
            tau = te.characteristic_time(allHams[j])
            Q_a = [0]*3
            redQ_a = [0]*3
            for i in range(3):
                Q_a[i] = (1/(2**(nVar - 1)))*te.time_evolution(tensor([sigma[i]] + I_env), allHams[j], tau)
                redQ_a[i] = Q_a[i].ptrace(0)
                redQ_a[i] = redQ_a[i].full()
            M_ab = [[(np.matmul(redQ_a[0], redQ_a[0])).trace(), (np.matmul(redQ_a[0], redQ_a[1])).trace(), (np.matmul(redQ_a[0], redQ_a[2])).trace()],
                    [(np.matmul(redQ_a[1], redQ_a[0])).trace(), (np.matmul(redQ_a[1], redQ_a[1])).trace(), (np.matmul(redQ_a[1], redQ_a[2])).trace()],
                    [(np.matmul(redQ_a[2], redQ_a[0])).trace(), (np.matmul(redQ_a[2], redQ_a[1])).trace(), (np.matmul(redQ_a[2], redQ_a[2])).trace()]]
            eig = np.linalg.eigh(M_ab)
            maxEigenValue = max(eig[0])
            score = (1/2) - (1/4)*maxEigenValue
            allScores.append(score)
        return max(allScores)
    else:
        return print('Error: entered OneorTwo value that is not 1 or 2.')

def CF2(thetasVar, nVar, hamiltonian, GGMMs, LUorU):
    '''
    Separate, simplified definition for use in cost minimization.
    '''
    allScores = []
    allTrM = []
    scramHamVar = scram(nVar, hamiltonian, thetasVar, GGMMs, LUorU)
    allHams = permuteHam(nVar, scramHamVar)
    sigma = [sigmax(), sigmay(), sigmaz()]
    I_env = [qeye(2)]*(nVar - 1)
    for j in range(len(allHams)):
        char_time = te.characteristic_time(allHams[j])
        Q_a = [0]*3
        redQ_a = [0]*3
        for i in range(3):
            Q_a[i] = (1/(2**(nVar - 1)))*te.time_evolution(tensor([sigma[i]] + I_env), allHams[j], char_time)
            redQ_a[i] = Q_a[i].ptrace(0)
            redQ_a[i] = redQ_a[i].full()
        M_ab = [[(np.matmul(redQ_a[0], redQ_a[0])).trace(), (np.matmul(redQ_a[0], redQ_a[1])).trace(), (np.matmul(redQ_a[0], redQ_a[2])).trace()],
                [(np.matmul(redQ_a[1], redQ_a[0])).trace(), (np.matmul(redQ_a[1], redQ_a[1])).trace(), (np.matmul(redQ_a[1], redQ_a[2])).trace()],
                [(np.matmul(redQ_a[2], redQ_a[0])).trace(), (np.matmul(redQ_a[2], redQ_a[1])).trace(), (np.matmul(redQ_a[2], redQ_a[2])).trace()]]
        eig = np.linalg.eigh(M_ab)
        maxEigenValue = max(eig[0])
        score = (1/2) - (1/4)*maxEigenValue
        allScores.append(score)
    return max(allScores)

def minimize(nVar, initial_hamVar, GGMMs):
    O = sci.optimize.minimize(CF2, randomThetas(0), (nVar, initial_hamVar, GGMMs, 0))
    minima = O.x.tolist()
    score = CF2(minima, nVar, initial_hamVar, GGMMs, 0)
    return score, minima

def minimizeCF1(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar):
    k = 0
    while k == 0:
        O = sci.optimize.minimize(cost, randomThetas(0), (nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 1))
        minima = O.x.tolist()
        score = cost(minima, nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 1)
        print(score)
        if abs(score) < 10**(-9):
            k = 1
            print('qcTPS found!')
        else:
            print('Did not find qcTPS.')
    return score, minima

def minimizeCF1bh(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar):
    k = 0
    while k == 0:
        O = sci.optimize.basinhopping(cost, randomThetas(0), niter = 100, minimizer_kwargs={'args':(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 1)})
        minima = O.x.tolist()
        score = cost(minima, nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 1)
        print(score)
        if abs(score) < 10**(-14):
            k = 1
            print('qcTPS found!')
        else:
            print('Did not find qcTPS.')
    return score, minima
    
def genericScore(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, numScores):
    meanList = []
    l = 0
    while l != numScores:
        score = cost(randomThetas(0), nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 1)
        #print(score)
        meanList.append(score)
        l += 1
    zeta = np.mean(meanList)
    epsilon = 0.01*zeta
    return zeta, epsilon

def TPSLines(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, numOrbits, numScores, minimizeYorN):
    orbitLabels = ['$H_n$']
    orbitScores = []
    meanOScores = []
    minOScores = []
    allTrM = []
    nullThetas = [0]*(dim_tot**2-1)
    # Adding native line first
    orbitScore = [cost(nullThetas, nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 0, 2)]
    for i in range(numScores):
        score = cost(randomThetas(1), nVar, initial_hamVar, GGMMs, tensor_initial_statesVar, 1, 1)
        orbitScore.append(score)
    orbitScores.append(orbitScore)
    if minimizeYorN == 0:
        # Adding arbitrary TPS Lines distinct from previous ones
        minTPSLines = [initial_hamVar]
        minTPSThetas = [nullThetas]
        l = 1
        while l != numOrbits:
            orbitLabels.append(f'$H_{l}$')         
            k = 2
            while k != 0: # a.k.a. while TPS line is not distinct
                scramHam = scram(nVar, initial_hamVar, randomThetas(0), GGMMs, 0)
                allHams = APHam(nVar, scramHam)
                print(f'Performing LUE check.')
                NCList = []
                for i in range(len(minTPSLines)):
                    for j in range(len(allHams)):
                        NCList.append(NC(nVar, allHams[j], minTPSLines[i]))
                if NCList == [0]*(len(allHams)*len(minTPSLines)):
                    print(f'Distinct TPS line found. Number of distinct TPS lines: {l+1}.')
                    k = 0
                else:
                    print('Ambiguous TPS line found. Trying again.')
                    k = 2
            minTPSLines.append(scramHam)
            orbitScore = [cost(nullThetas, nVar, scramHam, GGMMs, tensor_initial_statesVar, 0, 2)]
            for i in range(numScores):
                score = cost(randomThetas(1), nVar, scramHam, GGMMs, tensor_initial_statesVar, 1, 1)
                orbitScore.append(score)
            orbitScores.append(orbitScore)
            l += 1
        for i in range(len(orbitScores)):
            mean = np.mean(orbitScores[i])
            meanOScores.append(mean)
            minOScores.append(orbitScores[i][0])            
        return orbitLabels, orbitScores, minOScores, meanOScores, minTPSLines            
    if minimizeYorN == 1:
        # Adding minimum containing TPS lines distinct from previous ones
        minTPSLines = [initial_hamVar]
        minTPSThetas = [nullThetas]       
        l = 1
        while l != numOrbits:
            orbitLabels.append(f'$H_{l}$')
            k = 2
            while k != 0: # a.k.a. while TPS line is not distinct
                print(f'Minimizing...')
                #score, thetas = minimize(nVar, initial_hamVar, GGMMs)
                score, thetas = minimizeCF1(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar)
                #score, thetas = minimizeCF1bh(nVar, initial_hamVar, GGMMs, tensor_initial_statesVar)
                #print(score)
                scramHam = scram(nVar, initial_hamVar, thetas, GGMMs, 0)
                allHams = APHam(nVar, scramHam)                
                print(f'Done minimizing. Performing LUE check.')
                NCList = []
                for i in range(len(minTPSLines)):
                    for j in range(len(allHams)):
                        NCList.append(NC(nVar, allHams[j], minTPSLines[i]))
                if NCList == [0]*(len(allHams)*len(minTPSLines)):
                    print(f'Distinct TPS line found. Number of distinct TPS lines: {l+1}.')
                    k = 0
                else:
                    print('Ambiguous TPS line found. Trying again.')
                    k = 2
            minTPSLines.append(scramHam)
            minTPSThetas.append(thetas)
            orbitScore = [score]
            for i in range(numScores):
                score = cost(randomThetas(1), nVar, scramHam, GGMMs, tensor_initial_statesVar, 1, 1)
                orbitScore.append(score)
            orbitScores.append(orbitScore)
            l += 1            
        for i in range(len(orbitScores)):
            mean = np.mean(orbitScores[i])
            meanOScores.append(mean)
            minOScores.append(orbitScores[i][0])            
        return orbitLabels, orbitScores, minOScores, meanOScores, minTPSLines, minTPSThetas        
    else:
        return print('Error: entered minimize value that is not 0 or 1.')
    
def testPlot(orbitLabels, orbitScores, minOScores, meanOScores, zeta, epsilon, yub, labelPlace, title):
    # Setting up axes and labels
    fig1, ax1 = plt.subplots(figsize=(8, 8), dpi = 300)
    ax2 = ax1.twinx()
    ax1.set_ylim(bottom=y_lb, top=yub)
    ax1.set_xlabel('Tensor Product Structure', size = u)
    ax1.set_ylabel('Score', size = u)
    ax1.tick_params(axis='both', which='major', labelsize=20)
    ax2.set_ylim(bottom=y_lb, top=y_ub);
    ax2.set_yticks([]);
    plt.title(f'{title} ({n} qubits, {ham_name})', size=19)
    # Plotting data
    for xe, ye in zip(orbitLabels, orbitScores):
        ax1.scatter([xe] * len(ye), ye, **s2_args)
    for xe, ye in zip(orbitLabels, meanOScores):
        ax1.scatter(xe, ye, **s1M_args)
    for xe, ye in zip(orbitLabels, minOScores):
        ax1.scatter(xe, ye, **s2M_args)
    zetaLine = ax1.axhline(y=zeta, **ze_args)
    #epsilonLine = ax1.axhline(y=epsilon, **ze_args)
    epsilonLine = ax1.axhline(y=10**(-10), **ze_args)
    ax1.text(labelPlace, zeta-0.005,'$\mu_{TPS}$', fontsize="15")
    ax1.text(labelPlace, epsilon-0.005,'$\epsilon$', fontsize="15")
    handles, labels = ax1.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    leg = ax1.legend(by_label.values(), by_label.keys(), fontsize='xx-large', frameon=True)
    for lh in leg.legendHandles: 
        lh.set_alpha(1)
        
def selfHam(_n, _sc):
    _selfGGMMs = selfGGMMs(_n)
    _Hs = 0*tensor([qeye(2)]*_n)
    for i in range(len(_selfGGMMs)):
        _Hs += _sc[i]*_selfGGMMs[i]
    return _Hs

def nonSelfHam(_n, _nsc, _GGMMs):
    _nonSelfGGMMs = _GGMMs.copy()
    _selfGGMMs = selfGGMMs(_n)
    for x in _selfGGMMs:
        _nonSelfGGMMs.remove(x)
    _Hns = 0*tensor([qeye(2)]*_n)
    for i in range(len(_nonSelfGGMMs)):
        _Hns += _nsc[i]*_nonSelfGGMMs[i]
    return _Hns

## Plotting scores of TPS lines

In [ ]:
B = genericScore(n, initial_ham, totGGMMs, tensor_initial_states, 100)
#B = genericScore(n, initial_ham, totGGMMs, eSup, 100)
#B = genericScore(n, initial_ham, totGGMMs, randRho, 100)

In [ ]:
# Minimizing 
fiveDLinesList = []
l = 0
iterations = 1
while l != iterations:
    start = timeit.default_timer()
    fiveDLines = TPSLines(n, initial_ham, totGGMMs, tensor_initial_states, 5, 100, 1)
    #fiveDLines = TPSLines(n, initial_ham, totGGMMs, eSup, 5, 100, 1)
    #fiveDLines = TPSLines(n, initial_ham, totGGMMs, randRho, 2, 100, 1)
    end = timeit.default_timer()
    print(f'{l}: Took {end - start} s.')
    fiveDLinesList.append(fiveDLines)
    print('Done.')
    l += 1

In [ ]:
ind = 0
testPlot(*fiveDLinesList[ind][0:4], *B, 0.55, 4.3, 'Quasi-classical TPSs')

In [ ]:
minH = fiveDLinesList[0][4]
js = [] # coefficients
σs = [] # pauli bases
for i in range(len(minH)):
    tH = qml.pauli_decompose(minH[i].full())
    tHjs = []
    tHσs = []
    for k in range(len(tH.coeffs)):
        tHjs.append(tH.coeffs[k])
        tHσs.append(qml.pauli.pauli_word_to_string(tH.ops[k]))
    js.append(tHjs)
    σs.append(tHσs)

In [ ]:
# gives Hilbert-Schmidt norm of k-local terms
locCoeffs = []
for i in range(len(minH)):
    tσs = σs[i]
    tjs = js[i]
    hamLocCoeffs = [[]]*n
    for j in range(n):
        for l in range(len(tjs)):
            if tσs[l].count('I') == j:
                hamLocCoeffs[n-1-j] = [tjs[l]] + hamLocCoeffs[n-1-j]
            else:
                continue
    locCoeffs.append(hamLocCoeffs)

meanVals = []
for i in range(len(minH)):
    hamMeanVals = []
    tlocCoeffs = locCoeffs[i]
    for j in range(n):
        hamMeanVals.append(np.sqrt(np.sum(np.abs(locCoeffs[i][j])**2)))
    meanVals.append(hamMeanVals)
for i in range(len(minH)):
    norm = np.sum(meanVals[i])
    for j in range(len(meanVals[i])):
        meanVals[i][j] = meanVals[i][j]/norm
meanValsT = [list(x) for x in zip(*meanVals)]

In [ ]:
# Setting up axes and labels
fig1, ax1 = plt.subplots(figsize=(8, 8), dpi = 300)
ax2 = ax1.twinx()
#ax1.set_ylim(bottom=y_lb, top=yub)
ax1.set_xlabel('Tensor Product Structure', size = u)
ax1.set_ylabel('Hilbert-Schmidt Norm', size = u)
ax1.tick_params(axis='both', which='major', labelsize=20)
#ax2.set_ylim(bottom=y_lb, top=y_ub);
ax2.set_yticks([]);
plt.title(f'HS Norm of $k$-local Terms ({n} qubits, {ham_name})', size=19)
# Plotting data
for xe, ye in zip(fiveDLinesList[0][0], meanValsT[0]):
    ax1.scatter(xe, ye, **l1M_args)
for xe, ye in zip(fiveDLinesList[0][0], meanValsT[1]):
    ax1.scatter(xe, ye, **l2M_args)
#for xe, ye in zip(fiveDLinesList[0][0], meanVals[2]):
    #ax1.scatter(xe, ye, **l3M_args)
#for xe, ye in zip(fiveDLinesList[0][0], meanVals[3]):
    #ax1.scatter(xe, ye, **l4M_args)
ax1.set_ylim(bottom=-0.05, top=1.05)
handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax1.legend(by_label.values(), by_label.keys(), fontsize='xx-large', frameon=True, loc='best')

In [ ]:
# gives list of length n. the kth component tallies the number of k-local terms for the native, first, ... 
# qc-Hamiltonians
localNum = []
for i in range(n):
    kLocalNum = [0]*len(minH)
    for j in range(len(minH)):
        tσs = σs[j]
        for l in range(len(js[j])):
            if tσs[l].count('I') == i:
                kLocalNum[j] += 1
            else:
                continue
    localNum.append(kLocalNum)
localNum.reverse()